### Análise de eleição - Eduardo Paes (55)

In [1]:
import pandas as pd
import tse

In [2]:
database_1 = 'databases/rj_2024.sql'
database_2 = 'databases/eleicoes_2024.sql'

In [3]:
sql_data = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': 'rj'
    })

In [4]:
sql_data.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) rj
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [5]:
sql_data.create_index(
    table='resultados',
    columns=['ANO_ELEICAO', 'NM_VOTAVEL', 'NR_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)
sql_data.create_index(
    table='eleitorado',
    columns=['ANO_ELEICAO', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)

'Indices criados com sucesso'

In [6]:
CANDIDATO = 'EDUARDO PAES'

In [33]:
query = f"""
    SELECT 
        r.DS_CARGO as cargo, 
        r.NM_VOTAVEL as candidato, 
        r.NR_SECAO as secao, 
        l.NM_MUNICIPIO as municipio, 
        l.NM_BAIRRO as bairro,
        l.NR_LATITUDE as latitude,
        l.NR_LONGITUDE as longitude,
        e.DS_GENERO as genero,
        e.DS_RACA_COR as raca,
        e.DS_ESTADO_CIVIL as estado_civil,
        e.DS_FAIXA_ETARIA as faixa_etaria,
        e.DS_GRAU_ESCOLARIDADE as escolaridade,
        e.QT_ELEITORES_PERFIL as eleitores_perfil,
        l.QT_ELEITOR_SECAO as eleitores_secao,
        r.QT_VOTOS as votos_secao
    FROM 
        rj.resultados AS r
    INNER JOIN
        geral.locais AS l
        ON
            r.SG_UF = l.SG_UF
            AND r.CD_MUNICIPIO = l.CD_MUNICIPIO
            AND r.NR_ZONA = l.NR_ZONA
            AND r.NR_SECAO = l.NR_SECAO
    INNER JOIN 
        rj.eleitorado AS e
        ON 
            r.SG_UF = e.SG_UF
            AND r.CD_MUNICIPIO = e.CD_MUNICIPIO 
            AND r.NR_ZONA = e.NR_ZONA
            AND r.NR_LOCAL_VOTACAO = e.NR_LOCAL_VOTACAO
            AND r.NR_SECAO = e.NR_SECAO
    WHERE 
        r.NM_VOTAVEL LIKE '{'%'+ CANDIDATO.replace(' ', '%') + '%'}';
"""

In [ ]:
result = pd.read_sql_query(query, sql_data.conn)

In [ ]:
result.head(5)

In [13]:
def soma_votos_secao(grupo: pd.DataFrame):
    return grupo.drop_duplicates(subset='secao')['votos_secao'].sum()

def percentual(grupo: pd.DataFrame):
    votos_grupo = soma_votos_secao(grupo)
    return votos_grupo/grupo['eleitores_perfil'].sum()

In [ ]:
agrupar_por = 'bairro'

votos_bairro = result.groupby(agrupar_por).agg(
    eleitores_perfil=('eleitores_perfil', 'sum'),
    latitude=('latitude', 'mean'),
    longitude=('longitude', 'mean'),
    votos_secao=('votos_secao', lambda x: soma_votos_secao(result.loc[x.index])),
    percentual = ('votos_secao', lambda x: percentual(result.loc[x.index]))
).sort_values(by='percentual', ascending=False)
votos_bairro.head()

In [ ]:
agrupar_por = 'escolaridade'

votos_escola = result.groupby(agrupar_por).agg(
    eleitores_perfil=('eleitores_perfil', 'sum'),
    latitude=('latitude', 'mean'),
    longitude=('longitude', 'mean'),
    votos_secao=('votos_secao', lambda x: soma_votos_secao(result.loc[x.index])),
    percentual = ('votos_secao', lambda x: percentual(result.loc[x.index]))
).sort_values(by='percentual', ascending=False)
votos_escola.head()

In [ ]:
result['eleitores_perfil'].sum()

In [ ]:
por_secao = result.groupby('secao').agg({
    'eleitores_secao': 'max',
    'eleitores_perfil': 'sum'
})
por_secao.head()


In [ ]:
secao_254 = result[result['secao'] == 254]
secao_254['secao'].value_counts()

In [ ]:
secao_254.head()

In [ ]:
secao_254['eleitores_perfil'].sum()

In [31]:
query = """
    SELECT
        e.SG_UF AS uf,
        e.CD_MUNICIPIO AS municipio,
        e.NR_ZONA AS zona,
        e.NR_SECAO AS secao,
        SUM(e.QT_ELEITORES_PERFIL) AS eleitores_perfil,
        AVG(l.QT_ELEITOR_SECAO) AS eleitores_secao
    FROM
        rj.eleitorado AS e
    INNER JOIN
        geral.locais AS l
        ON 
            e.SG_UF = l.SG_UF
            AND e.CD_MUNICIPIO = l.CD_MUNICIPIO
            AND e.NR_ZONA = l.NR_ZONA
            AND e.NR_SECAO = l.NR_SECAO
    WHERE
        e.NR_SECAO = 254
    GROUP BY
        e.SG_UF, e.NR_ZONA, e.NR_SECAO
"""

In [32]:
result = pd.read_sql_query(query, sql_data.conn)
result.head()

,uf,municipio,zona,secao,eleitores_perfil,eleitores_secao
0,RJ,60011,4,254,366,366.0
1,RJ,60011,5,254,361,361.0
2,RJ,60011,9,254,396,396.0
3,RJ,60011,10,254,438,438.0
4,RJ,60011,16,254,350,350.0


In [26]:
query = f"""
    SELECT 
        r.DS_CARGO as cargo, 
        r.NM_VOTAVEL as candidato, 
        r.NR_SECAO as secao, 
        e.DS_GENERO as genero,
        SUM(e.QT_ELEITORES_PERFIL) as eleitores_perfil,
        MAX(l.QT_ELEITOR_SECAO) as eleitores_secao,
        AVG(r.QT_VOTOS) as votos_secao
    FROM 
        rj.resultados AS r
    INNER JOIN
        geral.locais AS l
        ON
            r.SG_UF = l.SG_UF
            AND r.CD_MUNICIPIO = l.CD_MUNICIPIO
            AND r.NR_ZONA = l.NR_ZONA
            AND r.NR_LOCAL_VOTACAO = l.NR_LOCAL_VOTACAO
            AND r.NR_SECAO = l.NR_SECAO
    INNER JOIN 
        rj.eleitorado AS e
        ON 
            r.SG_UF = e.SG_UF
            AND r.CD_MUNICIPIO = e.CD_MUNICIPIO 
            AND r.NR_ZONA = e.NR_ZONA
            AND r.NR_LOCAL_VOTACAO = e.NR_LOCAL_VOTACAO
            AND r.NR_SECAO = e.NR_SECAO
    WHERE 
        r.NM_VOTAVEL LIKE '{'%'+ CANDIDATO.replace(' ', '%') + '%'}'
    
    GROUP BY
        r.NM_VOTAVEL, r.NR_SECAO, e.DS_GENERO;

"""

In [ ]:
result = pd.read_sql_query(query, sql_data.conn)
result.head()

In [ ]:
result[result['secao'] == 254]['votos_secao'].sum()

In [48]:
query = """
    SELECT
        *
    FROM
        rj.resultados AS r
    
    WHERE
        r.NR_SECAO = 254
        AND r.NM_VOTAVEL LIKE '%EDUARDO%PAES%'
"""

In [ ]:
result = pd.read_sql_query(query, sql_data.conn)
result.head()

### Mapa de calor

In [27]:
import folium
from folium.plugins import HeatMap

In [28]:
mapa = folium.Map(location=[votos_bairro["NR_LATITUDE"].mean(), votos_bairro["NR_LONGITUDE"].mean()], zoom_start=12)

In [ ]:
heat_data = [[row["NR_LATITUDE"], row["NR_LONGITUDE"], row["PERCENTUAL"]] for _, row in votos_bairro.iterrows()]
HeatMap(heat_data).add_to(mapa)

In [ ]:
mapa.save("mapa_de_calor.html")
mapa